In [13]:
import pymongo
import time
import threading
import random

In [14]:
cl = pymongo.MongoClient("mongodb://localhost:27017")
db = cl["deep"]
col = db["flat"]

In [15]:
col.find_one()

{'_id': ObjectId('6866553db21c637ab5da3d6c'),
 'number_of_records': 999,
 'activity_sec': 93,
 'application': 'effect',
 'device': 767,
 'volume_total_bytes': 2606,
 'subscribers': 191}

In [28]:
indexes = col.list_indexes()
for i in indexes:
    print(i)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


In [16]:
def calc_perf(old, new):
    rounded_old = round(old)
    rounded_new = round(new)
    ratio = ((rounded_old - rounded_new) / rounded_old) * 100
    print(f"{rounded_old} {rounded_new} {ratio:.1f}")

In [17]:
def create_batch(batch_size):
    batch = []
    for _ in range(batch_size):
        el = {
            "num_of_records": random.randint(0,999),
            "activity_sec": random.randint(0,99),
            "application": "insert_batch_member",
            "device": random.randint(0,9999),
            "volume_total_bytes": random.randint(0,9999),
            "subscribers": random.randint(0,999),
        }
        batch.append(el)
    return batch

In [18]:
def measure_insert(user_id, batch_size, duration_list):
    duration = 0
    
    for _ in range(10):
        batch = create_batch(batch_size)
        start = time.perf_counter()
        col.insert_many(batch)
        end = time.perf_counter()
        
        duration += end - start
    
    # take average duration and convert to ms
    duration = (duration / 10) * 1000
    duration_list[user_id] = duration

In [19]:
def insert_test(num_of_users, batch_size):
    threads_list=[]
    duration_list = [None] * num_of_users
        
    batch_size = 10000
    for x in range(num_of_users):
        t = threading.Thread(target=measure_insert, args=(x, batch_size, duration_list,))
        t.start()
        threads_list.append(t)
    
    for t in threads_list:
        t.join()

    avg = (sum(duration_list) / len(duration_list))
    return avg

In [20]:
num_of_docs = 10000

one_user_old = insert_test(1, num_of_docs)

two_user_old = insert_test(2, num_of_docs)

four_user_old = insert_test(4, num_of_docs)

In [21]:
index_name = col.create_index([("device", 1)])

In [22]:
one_user_new = insert_test(1, num_of_docs)
two_user_new = insert_test(2, num_of_docs)
four_user_new = insert_test(4, num_of_docs)

In [23]:
calc_perf(one_user_old, one_user_new)
calc_perf(two_user_old, two_user_new)
calc_perf(four_user_old, four_user_new)

96 165 -71.9
146 210 -43.8
298 384 -28.9


In [24]:
col.count_documents({"application": "insert_batch_member"})

1400000

In [25]:
col.delete_many({"application": "insert_batch_member"})

DeleteResult({'n': 1400000, 'ok': 1.0}, acknowledged=True)

In [26]:
col.count_documents({})

500000

In [27]:
col.drop_index("device_1")